# Hi :)

----

# Import libraries

In [1]:
import numpy as np
import pandas as pd

----

These steps till next seperating line are for Kalman Filter implementation(run if you don't have KalmanFilter-SisFall.csv file)

----

----

# Loading the dataset

In [2]:
# # Loading the dataset
# SisFall_Dataset= pd.read_csv('Dataset/SisFall_dataset.csv')
# # Showing 5 random rows
# SisFall_Dataset.sample(5, random_state= 48)

In [3]:
# # Making a copy of original dataset to work with
# df= SisFall_Dataset.copy()

In [4]:
# # Checking the number of rows and columns
# df.shape

# Kalman Filter

Kalman Filter Implementation on three axis

Initialization

In [5]:
# # Creating a function to implement Kalman Filter
# def run_kalman_filter_vector(measurements, R_value, A, H, Q, x0, P0):
#     x= x0.copy()
#     P= P0.copy()
#     estimates= []

#     for z in measurements:
#         # Predict
#         x= A @ x
#         P= A @ P @ A.T + Q

#         # Update
#         y= np.array(z).reshape(-1, 1) - H @ x
#         S= H @ P @ H.T + R_value
#         K= P @ H.T @ np.linalg.inv(S)
#         x= x + K @ y
#         P= (np.eye(P.shape[0]) - K @ H) @ P

#         estimates.append(x.flatten())  # Store estimated state

#     return estimates

In [6]:
# A= np.eye(3)   # No dynamics for now
# Q= np.eye(3) * 0.001
# x0= np.zeros((3, 1))
# P0= np.eye(3)

# # Same H matrix as above
# H= np.array([
#     [1, 0, 0], [0, 1, 0], [0, 0, 1],  # accel1
#     [1, 0, 0], [0, 1, 0], [0, 0, 1],  # accel2
#     [1, 0, 0], [0, 1, 0], [0, 0, 1],  # gyro
# ])

# R= np.diag([0.05] * 6 + [0.1] * 3)  # More trust in accelerometers

In [7]:
# # Getting the measurements a specific order for Kalman Filter implementation
# measurements= df[['ADXL345_x', 'ADXL345_y', 'ADXL345_z', 'MMA8451Q_x', 'MMA8451Q_y', 'MMA8451Q_z', 'ITG3200_x', 'ITG3200_y', 'ITG3200_z']].values

Kalman Filter Main loop

In [8]:
# estimates= run_kalman_filter_vector(measurements, R, A, H, Q, x0, P0)

Adding Kalman Filter data to dataset

In [9]:
# # Adding Kalman Filter data
# df[['x_kalman', 'y_kalman', 'z_kalman']]=np.array(estimates).round(2)
# # Fixing the dataset columns
# df= df[[col for col in df.columns if col != 'Situation'] + ['Situation']]

In [10]:
# df.head()

In [11]:
# # Saving the dataset
# df.to_csv('Dataset/KalmanFilter-SisFall.csv', index= False)

----

----

If you have KalmanFilter-SisFall.csv file, you can work form here

----

# Loading the dataset

In [12]:
# Loading the dataset
KalmanFilter_SisFall= pd.read_csv('Dataset/KalmanFilter-SisFall.csv')
# Showing 5 random row of dataset
KalmanFilter_SisFall.sample(5, random_state= 48)

,ADXL345_x,ADXL345_y,ADXL345_z,ITG3200_x,ITG3200_y,ITG3200_z,MMA8451Q_x,MMA8451Q_y,MMA8451Q_z,x_kalman,y_kalman,z_kalman,Situation
636970,3,-237,-149,-120,-122,-96,-78,-881,-437,-52.55,-473.54,-247.41,Fall
734642,-22,59,-283,-43,18,-4,-181,259,-979,-89.62,131.23,-504.66,Fall
1069635,37,-348,-103,702,-766,437,133,-1329,-305,74.21,-828.16,-158.67,Not Fall
793919,279,41,-3,-28,46,-10,982,172,153,495.69,94.04,55.20,Fall
541529,7,-429,-144,-295,960,240,-64,-1627,-429,31.64,-598.62,-94.81,Fall


In [13]:
# Making a copy of original dataset to work with
df= KalmanFilter_SisFall.copy()

Sliding window

In [14]:
# Initialize the new column with empty lists
df['situation_ratio']= [[] for _ in range(len(df))]

window_size= 200
stride= 150

for start in range(0, len(df), stride):
    end= start + window_size
    if end > len(df):
        window_df= df['Situation'].iloc[start :]
    else:
        window_df= df['Situation'].iloc[start : end]
    
    win= window_df.value_counts().reset_index()
    win.columns= ['Situation', 'count']
    win.sort_values(by= 'count', ascending= True, inplace= True)

    # Make sure there are at least 2 classes to compare
    if len(win) > 1 and win['count'].values[1] != 0:
        value= int(win['count'].values[0] / win['count'].values[1] * 100)
    else:
        value= 0

    # Assign value to each row in this window
    for i in range(start, min(end, len(df))):
        df.at[i, 'situation_ratio'].append(value)

Siruation ration in every window

In [ ]:
# Lets check the ratio of situations in every window
exploded_df= df.copy()
exploded_df= exploded_df.explode(['situation_ratio'])

In [22]:
# Counting the different situation ratio in dataset
exploded_df['situation_ratio'].value_counts()

situation_ratio
0     2795950
60        200
Name: count, dtype: int64

Deleting rows with bad situation ratio

In [26]:
deleted_rows= exploded_df[exploded_df['situation_ratio'] != 0].index
df.drop(index= deleted_rows, inplace= True)

In [28]:
KalmanFilter_SisFall.shape

(2097150, 13)

In [27]:
df.shape

(2096950, 14)

In [29]:
KalmanFilter_SisFall.shape[0] - df.shape[0]

200

In [ ]:
# Checking the last window
(df.shape[0] - 150) / 200

10484.0

In [36]:
# Deleting the last 150 rows(for getting all perfect windows with 200 rows)
df.drop(index= [i for i in range(len(df) - 150, len(df))], inplace= True)

In [39]:
df.shape[0] / 200

10484.0

In [40]:
df.sample(5, random_state= 48)

,ADXL345_x,ADXL345_y,ADXL345_z,ITG3200_x,ITG3200_y,ITG3200_z,MMA8451Q_x,MMA8451Q_y,MMA8451Q_z,x_kalman,y_kalman,z_kalman,Situation,situation_ratio
1233484,256,38,24,-46,22,4,990,145,224,490.93,77.85,100.53,Not Fall,"[0, 0]"
1462488,19,-155,190,35,10,-11,59,-655,806,39.96,-322.30,395.71,Not Fall,[0]
437190,-151,141,-651,-3101,-1123,484,-599,1032,-2679,-940.20,-215.04,-1182.59,Fall,[0]
1059586,72,-361,-5,-555,298,-370,267,-1380,108,30.23,-565.72,60.65,Not Fall,[0]
1660984,89,-146,-275,-514,-449,339,216,-492,-946,-159.90,-283.34,-153.79,Not Fall,"[0, 0]"
